# AZ Cardinals Project

ETL NFL boxscore data for Arizona Cardinals games.

## Imports

In [13]:
import pandas as pd
import json
import os
import requests

from dotenv import load_dotenv

## Setup API Authentication

In [4]:
#API Endpoint for NFL Box Scores
url = "https://tank01-nfl-live-in-game-real-time-statistics-nfl.p.rapidapi.com/getNFLBoxScore"

In [9]:
#Get API Key and host
load_dotenv()
api_token = os.getenv('nfl_api_key')
api_host = os.getenv('rapid_api_host')

In [10]:
#rapidapi headers
headers = {
	"X-RapidAPI-Key": "{key}".format(key=api_token),
	"X-RapidAPI-Host": "{host}".format(host=api_host)
}

### Pull Single Game Data - Test

#### Set QueryString

In [27]:
#setup querystring to pull in the exciting 9/18 game against the Raiders (one of our few wins....)
querystring = {"gameID":"20220918_ARI@LV","fantasyPoints":"false"}

#### Call API and Review Results

In [28]:
response = requests.get(url, headers=headers, params=querystring)

In [29]:
print(json.dumps(response.json(), indent=2)) #pretty print

{
  "statusCode": 200,
  "body": {
    "playerStats": {
      "2991662": {
        "gameID": "20220918_ARI@LV",
        "Receiving": {
          "receptions": "5",
          "recTD": "0",
          "longRec": "23",
          "targets": "8",
          "recYds": "66",
          "recAvg": "13.2"
        },
        "playerName": "Mack Hollins",
        "teamID": "17",
        "team": "LV",
        "teamAbv": "LV",
        "Defense": {
          "totalTackles": "1",
          "defTD": "0",
          "soloTackles": "1",
          "tfl": "0",
          "qbHits": "0",
          "sacks": "0",
          "passDeflections": "0"
        },
        "playerID": "2991662"
      },
      "2971816": {
        "gameID": "20220918_ARI@LV",
        "playerName": "Nick Vigil",
        "teamID": "1",
        "team": "ARI",
        "teamAbv": "ARI",
        "Defense": {
          "totalTackles": "5",
          "defTD": "0",
          "soloTackles": "3",
          "tfl": "1",
          "qbHits": "0",
         

#### Export Test Json

In [32]:
with open('../data/test_boxscore.json', 'w') as file:
    json.dump(response.json(), file)

### Save to DataFrame

Create a table of the game details combined with the linescore and some summary team stats

In [38]:
boxscore = response.json()['body']

In [44]:
boxscore['lineScore']['away']['Q1']

'0'

In [52]:
game_summary = {
    "game_id": boxscore['gameID'],
    "game_type": boxscore['seasonType'],
    "game_date_id": boxscore['gameDate'],
    "away_team_id": boxscore['teamIdAway'],
    "away_team": boxscore['away'],
    "away_q1_score": boxscore['lineScore']['away']['Q1'],
    "away_q2_score": boxscore['lineScore']['away']['Q2'],
    "away_q3_score": boxscore['lineScore']['away']['Q3'],
    "away_q4_score": boxscore['lineScore']['away']['Q4'],
    "away_ot_score": boxscore['lineScore']['away']['OT'],
    "away_total_score": boxscore['lineScore']['away']['totalPts'],
    "away_total_plays": boxscore['teamStats']['away']['totalPlays'],
    "away_total_yards": boxscore['teamStats']['away']['totalYards'],
    "away_passing_yards": boxscore['teamStats']['away']['passingYards'],
    "away_rushing_yards": boxscore['teamStats']['away']['rushingYards'],
    "away_turnovers": boxscore['teamStats']['away']['turnovers'],
    "away_time_of_possession": boxscore['teamStats']['away']['possession'],
    "away_result": boxscore['awayResult'],
    "home_team_id": boxscore['teamIDHome'],
    "home_team": boxscore['home'],
    "home_q1_score": boxscore['lineScore']['home']['Q1'],
    "home_q2_score": boxscore['lineScore']['home']['Q2'],
    "home_q3_score": boxscore['lineScore']['home']['Q3'],
    "home_q4_score": boxscore['lineScore']['home']['Q4'],
    "home_ot_score": boxscore['lineScore']['home']['OT'],
    "home_total_score": boxscore['lineScore']['home']['totalPts'],
    "home_total_plays": boxscore['teamStats']['home']['totalPlays'],
    "home_total_yards": boxscore['teamStats']['home']['totalYards'],
    "home_passing_yards": boxscore['teamStats']['home']['passingYards'],
    "home_rushing_yards": boxscore['teamStats']['home']['rushingYards'],
    "home_turnovers": boxscore['teamStats']['home']['turnovers'],
    "home_time_of_possession": boxscore['teamStats']['home']['possession'],
    "home_result": boxscore['homeResult']
}

In [53]:
game_summary

{'game_id': '20220918_ARI@LV',
 'game_type': 'Regular Season',
 'game_date_id': '20220918',
 'away_team_id': '1',
 'away_team': 'ARI',
 'away_q1_score': '0',
 'away_q2_score': '0',
 'away_q3_score': '7',
 'away_q4_score': '16',
 'away_ot_score': '6',
 'away_total_score': '29',
 'away_total_plays': '78',
 'away_total_yards': '413',
 'away_passing_yards': '270',
 'away_rushing_yards': '143',
 'away_turnovers': '1',
 'away_time_of_possession': '36:22',
 'away_result': 'W',
 'home_team_id': '17',
 'home_team': 'LV',
 'home_q1_score': '7',
 'home_q2_score': '13',
 'home_q3_score': '3',
 'home_q4_score': '0',
 'home_ot_score': '0',
 'home_total_score': '23',
 'home_total_plays': '61',
 'home_total_yards': '324',
 'home_passing_yards': '244',
 'home_rushing_yards': '80',
 'home_turnovers': '1',
 'home_time_of_possession': '29:47',
 'home_result': 'L'}